## Анализ emotion skill в случае плохого настроения, подсчет среднего количества liked/disliked entities за диалог и составление топа

In [1]:
import json

In [29]:
import collections

In [327]:
recommended_data = []

for i in [19, 20]:
    data = json.load(open("Downloads/alexaprize-export-" + str(i) + ".json", "r"))
    clear_data = []
    for d in data:
        if d['utterances'][0]['attributes'].get('version', "") in ['v14.5.0', 'v14.4.0', 'v14.6.0', 'v14.7.0', 'v14.9.1']:
            clear_data.append(d)
    recommended_data += clear_data
    
#not_recommended_data = json.load(open("Downloads/alexaprize-export-10.json", "r")

### Emotion skill 

In [107]:
for d in recommended_data:
    for i, el in enumerate(d['utterances']):
        if len(el.get("active_skill", "")) > 0 and el["active_skill"] == "emotion_skill" and i - 2 >= 0 and i + 4 <= len(d):
            text = [utt["text"] for utt in d['utterances'][i-2:i+4]]
            print(text)
            print("___________")
            break

['Hi, this is an Alexa Prize Socialbot! How are you?', 'tell me a joke', 'Hey, I love telling jokes but not during this competition. Dead serious about that.', 'no', 'Daniel, I guess I am out of jokes, sorry. Do you know any good jokes?', 'tell me a joke']
___________
['Hi, this is an Alexa Prize Socialbot! How are things?', 'terrible', 'Please calm down. Frequent feelings of displeasure have been linked to higher risk of heart disease, high blood pressure, depression and difficulty sleeping. When I feel like you, I usually recall some funny jokes. Do you want to hear one?', 'yes', 'What is the preferred food of runners? - Fast food.', 'ha-ha ha']
___________
['Hi, this is an Alexa Prize Socialbot! How are you doing today?', 'horrible', 'I am sorry to hear that! I know a few ways to cheer up. Can I tell you a joke?', 'no', 'Do you think that some advices might help? I know a lot.', 'tell me about greninja']
___________
['Hi, this is an Alexa Prize Socialbot! How are things?', 'horrible

In [121]:
bad_emotions = ['anger', 'fear', 'sadness']

for d in recommended_data:
    for i, el in enumerate(d['utterances']):
        current_emotions = el.get("annotations", {}).get('combined_classification', {}).get("emotion_classification", {})
        user_bad_emotions = set(current_emotions) & set(bad_emotions)
        if len(user_bad_emotions) > 0 and i - 2 >= 0 and i + 4 <= len(d):
            text = [utt["text"] for utt in d['utterances'][i-2:i+4]]
            for t in text:
                print(t)
            print("___________")
            print(el["text"])
            print("___________")

i just need to talk to you about something
Hi, this is an Alexa Prize Socialbot! How are you?
and i was like
Aha. What are we gonna talk about?
huh i got he's really sick and just wasn't doing well
Well. While I can't tell you about my home soccer team, I'm very proud of it. Do you like fitness?
___________
and i was like
___________
let's chat
Jianked Carti, Hi, this is an Alexa Prize Socialbot! How are you doing today?
my mom got dumbo
What do you want to talk about?
you know
Okay, Let's chat about France. okay, i thought maybe you were from america and that french fries weren ' t actually made in france.
___________
my mom got dumbo
___________
let's have a conversation
Hi, this is an Alexa Prize Socialbot! What do you do on weekdays?
school
Are you a school student now?
yes
So, pupil, yeah. That's a great time, don't miss that time! Do you like your school?
___________
school
___________
let's chat
Hi, this is an Alexa Prize Socialbot! What do you do on weekdays?
read
When you have

In [134]:
bad_emotions = ["bored", "horrible", "terrible", "not good", "bad", "sad", 
                "a bit down", "not well", "shitty", "scared"]

count = 0

for d in recommended_data:
    for i, el in enumerate(d['utterances']):
        if i - 2 >= 0 and i + 4 <= len(d) and i % 2 == 0:
            for em in bad_emotions:
                if em in el["text"]:
                    text = [utt["text"] for utt in d['utterances'][i-2:i+4]]
                    for t in text:
                        print(t)
                    print("___________")
                    print(d['utterances'][i+1]["active_skill"])
                    print("____________")
                    count += 1

talk to me
Hi, this is an Alexa Prize Socialbot! How are you?
i'm horrible how are you
I'm living the dream. Do you want to know what I can do?
yes
I'm a socialbot running inside Alexa, and I'm all about chatting with people like you. I can answer questions, share fun facts, discuss movies, books and news. What do you wanna talk about? How about music or games?
___________
dff_friendship_skill
____________
let's chat
Hi, this is an Alexa Prize Socialbot! How are things?
terrible
Please calm down. Frequent feelings of displeasure have been linked to higher risk of heart disease, high blood pressure, depression and difficulty sleeping. When I feel like you, I usually recall some funny jokes. Do you want to hear one?
yes please
I can’t believe I forgot to go to the gym today. That’s 7 years in a row now.
___________
emotion_skill
____________
can i have a real conversation with you
Hi, this is an Alexa Prize Socialbot! How is the day going so far for you?
terrible
Please calm down. Freque

In [135]:
count

65

### Human entities

In [290]:
entitites = []
all_entities = []
dislike_entities = []
human_attitudes = {'like': 0, 'dislike': 0}
count = []
like_count = []
dislike_count = []
count_names = []
no_attitude = []
current_count = []

for d in recommended_data:
    all_entities += list(d["human"]['attributes']['entities'].keys())
    current_count.append(len(list(d["human"]['attributes']['entities'].keys())))
    current_like_count = 0
    current_dislike_count = 0
    for k in d["human"]['attributes']['entities'].values():
        if k.get('human_attitude', ""):
            human_attitudes[k['human_attitude']] += 1
            if k['human_attitude'] == 'like':
                entitites.append(k["name"])
                current_like_count += 1
            if k['human_attitude'] == 'dislike':
                dislike_entities.append(k["name"])
                current_dislike_count += 1
        else:
            no_attitude.append(k["name"])
    like_count.append(current_like_count)
    dislike_count.append(current_dislike_count)

In [294]:
len(dislike_entities)

10

In [296]:
len(all_entities) - len(entitites) - len(dislike_entities)

901

In [286]:
dislike_entities

['beyonce', 'drug', 'horseback']

In [279]:
sum(like_count)/len(like_count)

0.0015923566878980893

In [280]:
zero_count = [el for el in current_count if el == 0]

In [281]:
non_zero_count = [el for el in current_count if el != 0]

In [282]:
sum(non_zero_count)/len(non_zero_count)

3.8771428571428572

In [269]:
len(zero_count)

278

In [59]:
len(entitites)

515

In [68]:
sum(non_zero_count)/len(non_zero_count)

2.4044444444444446

In [283]:
c = collections.Counter()
for word in entitites:
    c[word] += 1

In [284]:
c

Counter({'water': 1})

In [285]:
c.most_common(20)

[('water', 1)]

In [24]:
recommended_data[10]["human"]['attributes']['entities']

{'pizza': {'name': 'pizza',
  'bot_attitude': None,
  'bot_encounters': [],
  'human_attitude': 'like',
  'human_encounters': [{'full_name': 'pizza',
    'previous_skill_name': 'dff_food_skill',
    'human_utterance_index': 4}]}}